# Song Recommendation System

In [1]:
# General tools
import os
import datetime

#For data Handling
import pandas as pd
import numpy as np

#For Visualizations
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 
%matplotlib inline

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

#Progreebar
from tqdm import tqdm

# For transformations and predictions
from scipy.optimize import curve_fit
from yellowbrick.target import FeatureCorrelation
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# For scoring
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score,mean_absolute_error

# For validation
from sklearn.model_selection import train_test_split

In [3]:
#Load the dataset
df = pd.read_csv('https://raw.githubusercontent.com/Sairaghav1999/Dataset-final/master/data.csv?token=AG3SCJ2FGGMLHCIEOSK3K5LBYZ2ZQ')

#Remove the Square Brackets from the artists

df["artists"]=df["artists"].str.replace("[", "")
df["artists"]=df["artists"].str.replace("]", "")
df["artists"]=df["artists"].str.replace("'", "")

def normalize_column(col):
    """
    col - column in the dataframe which needs to be normalized
    """
    max_d = df[col].max()
    min_d = df[col].min()
    df[col] = (df[col] - min_d)/(max_d - min_d)
    
#Normalize allnumerical columns so that min value is 0 and max value is 1
num_types = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
num = df.select_dtypes(include=num_types)
        
for col in num.columns:
    normalize_column(col)
    
#perform Kmeans CLustering
km = KMeans(n_clusters=25)
pred = km.fit_predict(num)
df['pred'] = pred
normalize_column('pred')

#Song Recommender
class Song_Recommender():
   
    def __init__(self, data):
        self.data_ = data
    
    #function which returns recommendations, we can also choose the amount of songs to be recommended
    def get_recommendations(self, song_name, n_top):
        distances = []
        #choosing the given song_name and dropping it from the data
        song = self.data_[(self.data_.name.str.lower() == song_name.lower())].head(1).values[0]
        rem_data = self.data_[self.data_.name.str.lower() != song_name.lower()]
        for r_song in tqdm(rem_data.values):
            dist = 0
            for col in np.arange(len(rem_data.columns)):
                #indeces of non-numerical columns(id,Release date,name,artists)
                if not col in [3,8,14,16]:
                    #calculating the manhettan distances for each numerical feature
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
        rem_data['distance'] = distances
        #sorting our data to be ascending by 'distance' feature
        rem_data = rem_data.sort_values('distance')
        columns = ['artists', 'name']
        return rem_data[columns][:n_top]


#Instantiate recommender class
recommender = Song_Recommender(df)

#Get recommendations 'Red Roses (feat. Landon Cube)' song
recommender.get_recommendations('Red Roses (feat. Landon Cube)', 5)

100%|██████████| 170652/170652 [00:13<00:00, 12537.69it/s]


,artists,name
57027,"Vince Staples, 6LACK, Mereba","Yo Love - From ""Queen & Slim: The Soundtrack"""
38257,"Post Malone, Halsey, Future",Die For Me (feat. Future & Halsey)
107211,The Weeknd,Initiation
91059,"Baby Bash, Z-Ro, Berner, Baby E",Light Up Light Up
56598,Post Malone,Hit This Hard


In [4]:
recommender.get_recommendations('dynamite', 10)

100%|██████████| 170646/170646 [00:12<00:00, 13666.21it/s]


,artists,name
36728,Bowling For Soup,Stacy's Mom
18017,One Direction,What Makes You Beautiful
54538,Billy Talent,Fallen Leaves
107011,Glee Cast,Loser Like Me (Glee Cast Version)
17818,Hot Chelle Rae,Tonight Tonight
91698,Paramore,Rose-Colored Boy
18192,Katy Perry,Part Of Me
37542,5 Seconds of Summer,She's Kinda Hot
54360,"RBD, Anahí, Dulce María, Maite Perroni, Christ...",Aún Hay Algo
124309,Easton Corbin,A Girl Like You
